In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.strtree import STRtree

In [ ]:
london_gdf = gpd.read_file("../data/GreatLondonShp/GreatLondonShp.shp")
openmapping_gdf = gpd.read_file("../data/ssx_openmapping_gb_v1.geojson")

In [2]:
fp_open = "../data/ssx_openmapping_gb_v1.geojson"
fp_ldn  = "../data/GreatLondonShp/GreatLondonShp.shp"

open_gdf = gpd.read_file(fp_open)
ldn_gdf  = gpd.read_file(fp_ldn)

print("open CRS:", open_gdf.crs, "bounds:", open_gdf.total_bounds)
print("ldn  CRS:", ldn_gdf.crs,  "bounds:", ldn_gdf.total_bounds)

# —— 关键：如果 OpenMapping 被读成 None/4326，但 bounds 看起来像米级（几万到几十万），强制设为 27700
if (open_gdf.crs is None or str(open_gdf.crs).endswith("4326")):
    # 粗判：若 x/y 绝对值大于 1000，基本就是米（27700）
    minx, miny, maxx, maxy = open_gdf.total_bounds
    if max(abs(minx), abs(miny), abs(maxx), abs(maxy)) > 1000:
        open_gdf = open_gdf.set_crs(27700, allow_override=True)

# 统一到 27700
ldn_27700  = ldn_gdf.to_crs(27700)
open_27700 = open_gdf.to_crs(27700)

# 用 dissolve 得到单一边界，并用 buffer(0) 修复几何
ldn_boundary = ldn_27700.dissolve().buffer(0)

# 裁剪（或先做一次空间连接预筛以提速也行）
london_centrality = gpd.clip(open_27700, ldn_boundary)

print("rows after clip:", len(london_centrality))
london_centrality.to_file("../data/s3/ssx_openmapping_london.geojson", driver="GeoJSON")


open CRS: EPSG:4326 bounds: [  62428.    8138.  655558. 1216985.]
ldn  CRS: EPSG:4326 bounds: [-0.5103 51.2868  0.334  51.6919]
rows after clip: 115418


In [ ]:
import geopandas as gpd

# 伦敦边界
london_gdf = gpd.read_file("../data/GreatLondonShp/GreatLondonShp.shp")

# OpenMapping GPKG（读取完整字段名）
openmapping_gdf = gpd.read_file("../data/ssx_openmapping_gb_v1.gpkg")

# 统一 CRS（通常都是 EPSG:27700，但保险起见）
london_gdf = london_gdf.to_crs(openmapping_gdf.crs)

# 裁剪
london_centrality_gdf = gpd.clip(openmapping_gdf, london_gdf)

# 导出
london_centrality_gdf.to_file("../data/s3/ssx_openmapping_london_gpkg.geojson", driver="GeoJSON")
